In [1]:
import torch
import os

print(f"PyTorch Version: {torch.__version__}")
print(f"Current Directory: {os.getcwd()}")
print(f"Files in Directory: {os.listdir('.')}")


PyTorch Version: 2.6.0+cu124
Current Directory: C:\Users\arche\PycharmProjects\yt-viral-predictor
Files in Directory: ['.git', '.gitignore', '.idea', '.ipynb_checkpoints', '.venv', 'api.txt', 'data', 'docker-compose.gpu.yml', 'docker-compose.yml', 'Dockerfile', 'download.py', 'LICENSE', 'main.py', 'nb.ipynb', 'README.md', 'requirements.gpu.txt', 'requirements.txt', 'thumbnails']


In [ ]:
print(torch.cuda.is_available())

In [2]:
with open("api.txt", "r") as f:
    api_key = f.read().strip()


import os
import pandas as pd

import googleapiclient.discovery
import googleapiclient.errors

from IPython.display import JSON

scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

def main(id_list):
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey=api_key)

    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(id_list)
    )
    response = request.execute()

    return JSON(response)

channel_ids = [
    'UCgKFOz_KrMbmypWrawtzDQg',
]

display(main(channel_ids))

<IPython.core.display.JSON object>

In [3]:
import requests
from googleapiclient.discovery import build

API_KEY = api_key
VIDEO_ID = "3fpqMJCoUQ0" # Example ID

def get_thumbnail_api(video_id, api_key):
    youtube = build("youtube", "v3", developerKey=api_key)

    # Request video details
    request = youtube.videos().list(
        part="snippet",
        id=video_id
    )
    response = request.execute()

    # Extract the highest resolution URL available
    thumbnails = response['items'][0]['snippet']['thumbnails']

    # Try to get maxres, then high, then default
    if 'maxres' in thumbnails:
        return thumbnails['maxres']['url']
    return thumbnails.get('high', {}).get('url')

thumbnail_url = get_thumbnail_api(VIDEO_ID, API_KEY)
print(f"Found URL: {thumbnail_url}")


def download_image(url, filename="thumbnail.jpg"):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Success! Image saved as {filename}")
    else:
        print("Failed to download image.")


Found URL: https://i.ytimg.com/vi/3fpqMJCoUQ0/maxresdefault.jpg


In [ ]:
download_image(thumbnail_url, "data/raw/thumbnails/test.jpg")

In [4]:
from googleapiclient.discovery import build

PLAYLIST_ID = "PLcEN1HW6GYIczNHmEUSKkJ646Ev3-i0A2"

def get_video_ids_from_playlist(playlist_id, api_key):
    youtube = build("youtube", "v3", developerKey=api_key)
    video_ids = []
    next_page_token = None

    while True:
        # Request a page of playlist items
        request = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        )
        response = request.execute()

        # Extract Video IDs from the current page
        for item in response.get('items', []):
            video_ids.append(item['contentDetails']['videoId'])

        # Check if there's another page
        next_page_token = response.get('nextPageToken')

        if not next_page_token:
            break

    return video_ids

# Usage
all_ids = get_video_ids_from_playlist(PLAYLIST_ID, API_KEY)
print(f"Retrieved {len(all_ids)} video IDs.")
print(all_ids[:5])  # Show first five

Retrieved 12 video IDs.
['xx4vDZdRWDU', 'CHzha4Fxlg0', 'JGWabJWC24U', 'gaKUpWq5Qas', 'HWRZRfmrxRc']


In [ ]:
for id in all_ids:
    download_image(get_thumbnail_api(id, API_KEY), f"data/raw/thumbnails/{id}.jpg")

In [ ]:
from googleapiclient.discovery import build

CHANNEL_ID = channel_ids[0]

def get_all_video_ids(channel_id, api_key):
    youtube = build("youtube", "v3", developerKey=api_key)

    # STEP 1: Get the 'uploads' playlist ID
    ch_request = youtube.channels().list(
        part="contentDetails",
        id=channel_id
    )
    ch_response = ch_request.execute()

    # The ID typically looks like the channel ID but starts with 'UU' instead of 'UC'
    uploads_playlist_id = ch_response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

    # STEP 2: Iterate through the playlist items
    video_ids = []
    next_page_token = None

    while True:
        pl_request = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=uploads_playlist_id,
            maxResults=50,
            pageToken=next_page_token
        )
        pl_response = pl_request.execute()

        for item in pl_response.get('items', []):
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = pl_response.get('nextPageToken')
        if not next_page_token:
            break

    return video_ids

# Usage
ids = get_all_video_ids(CHANNEL_ID, API_KEY)
print(f"Found {len(ids)} videos.")

In [9]:
import os
import requests
import yt_dlp
from tqdm import tqdm

# --- CONFIGURATION ---
# API_KEY = api_key
CHANNEL_ID = channel_ids[0]
OUTPUT_FOLDER = "data/raw/thumbnails"

# Ensure output directory exists
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

def get_long_form_videos(channel_id):
    """
    Uses yt-dlp to scrape ONLY the 'Videos' tab (Long-form).
    This automatically excludes anything YouTube considers a 'Short'.
    """
    # We specifically target the /videos endpoint
    channel_url = f"https://www.youtube.com/channel/{channel_id}/videos"

    print(f"Fetching long-form list from: {channel_url}")

    ydl_opts = {
        'quiet': True,
        'extract_flat': True, # FAST: Only grabs metadata, doesn't download video
        'sleep_interval': 1,  # Be polite to avoid blocking
    }

    long_form_videos = []

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            # Extract info from the specific tab
            info = ydl.extract_info(channel_url, download=False)

            # The 'entries' key contains the videos
            if 'entries' in info:
                # We convert the generator to a list to count them
                for entry in info['entries']:
                    long_form_videos.append({
                        'id': entry['id'],
                        'title': entry['title'],
                        # yt-dlp usually finds the best thumbnail automatically
                        'thumbnail': entry.get('thumbnails', [{}])[-1].get('url')
                    })
        except Exception as e:
            print(f"Error fetching data: {e}")

    return long_form_videos

def download_thumbnail(video):
    """Downloads the thumbnail for a single video."""
    try:
        vid_id = video['id']
        url = video['thumbnail']

        # If yt-dlp didn't give a direct URL, construct the maxres one
        if not url:
            url = f"https://img.youtube.com/vi/{vid_id}/maxresdefault.jpg"

        response = requests.get(url, timeout=10)

        if response.status_code == 200:
            filename = os.path.join(OUTPUT_FOLDER, f"{vid_id}.jpg")
            with open(filename, "wb") as f:
                f.write(response.content)
            return True
    except:
        pass
    return False

def main():
    # 1. Get the Clean List (Long-form only)
    videos = get_long_form_videos(CHANNEL_ID)
    print(f"Found {len(videos)} videos in the 'Videos' tab.")

    # 2. Download Images
    print("Downloading thumbnails...")
    success_count = 0

    # Using a simple loop with progress bar (Threads usually not needed for <1000 items here)
    for video in tqdm(videos):
        if download_thumbnail(video):
            success_count += 1

    print(f"\nDone! Saved {success_count} thumbnails to '{OUTPUT_FOLDER}'")

if __name__ == "__main__":
    main()

In [ ]:
OUTPUT_FOLDER = "data/raw/thumbnails"
DATA_FILE = "data/raw/video_data.csv"

In [11]:
import os
import csv
import requests
import yt_dlp
import statistics
from tqdm import tqdm

# --- CONFIGURATION ---
OUTPUT_FOLDER = "data/raw/thumbnails"
DATA_FILE = "data/raw/video_data.csv"

# Ensure output directory exists
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

def get_long_form_data(channel_id):
    channel_url = f"https://www.youtube.com/channel/{channel_id}/videos"
    print(f"Fetching metadata from: {channel_url}...")

    ydl_opts = {
        'quiet': True,
        'extract_flat': True,
        'sleep_interval': 1,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            info = ydl.extract_info(channel_url, download=False)
            if 'entries' in info:
                return list(info['entries'])
        except Exception as e:
            print(f"Error fetching data: {e}")
            return []
    return []

def calculate_rolling_average(all_videos, current_index, window=10):
    """
    Calculates avg views of previous videos.
    Returns None if there is NO history (first video).
    """
    start_idx = current_index + 1
    end_idx = start_idx + window

    # Get previous videos (slice handles out-of-bounds automatically)
    previous_videos = all_videos[start_idx : end_idx]

    # Filter valid view counts
    views = [v.get('view_count') for v in previous_videos if v.get('view_count') is not None]

    if not views:
        return None # No history available (First video)

    return statistics.mean(views)

def download_thumbnail(video_id, url):
    try:
        if not url:
            url = f"https://img.youtube.com/vi/{video_id}/maxresdefault.jpg"
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            with open(os.path.join(OUTPUT_FOLDER, f"{video_id}.jpg"), "wb") as f:
                f.write(response.content)
    except:
        pass

def main():
    videos = get_long_form_data(CHANNEL_ID)
    print(f"Found {len(videos)} videos. Processing...")

    with open(DATA_FILE, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        # Updated Header: No Thumbnail File, Added VScore
        writer.writerow(['Video ID', 'Title', 'Current Views', 'Avg Views (Last 10)', 'Virality Score'])

        for i, video in tqdm(enumerate(videos), total=len(videos)):
            vid_id = video.get('id')
            title = video.get('title')
            current_views = video.get('view_count')
            thumb_url = video.get('thumbnails', [{}])[-1].get('url')

            if not vid_id or current_views is None:
                continue

            # 1. Calculate Stats
            avg_prev = calculate_rolling_average(videos, i, window=10)

            # 2. Calculate Virality Score
            # Formula: Current / Average
            if avg_prev and avg_prev > 0:
                v_score = current_views / avg_prev
                avg_str = f"{avg_prev:.0f}"
                score_str = f"{v_score:.2f}" # e.g., 1.50 (1.5x better than avg)
            else:
                # Handle the "First Video" or "Zero History" case
                avg_str = "N/A"
                score_str = "N/A"

            # 3. Download Thumbnail (Image only)
            download_thumbnail(vid_id, thumb_url)

            # 4. Save Data
            writer.writerow([
                vid_id,
                title,
                current_views,
                avg_str,
                score_str
            ])

    print(f"\nDone! Data saved in '{DATA_FILE}'")

if __name__ == "__main__":
    main()

In [5]:
import os
import csv
import requests
import yt_dlp
import statistics
from tqdm import tqdm

# --- CONFIGURATION ---
NEIGHBOR_COUNT = 4  # Total neighbors to analyze (5 before + 5 after)
OUTPUT_FOLDER = "data/raw/thumbnails"
CHANNEL_ID = 'UCgKFOz_KrMbmypWrawtzDQg'


if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

def get_long_form_data(channel_id):
    """Fetch all video metadata."""
    channel_url = f"https://www.youtube.com/channel/{channel_id}/videos"
    print(f"Fetching metadata from: {channel_url}...")

    ydl_opts = {
        'quiet': True,
        'extract_flat': True,
        'sleep_interval': 1,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            info = ydl.extract_info(channel_url, download=False)
            if 'entries' in info:
                return list(info['entries'])
        except Exception as e:
            print(f"Error fetching data: {e}")
            return []
    return []

def calculate_elastic_average(all_videos, current_index, neighbor_target=10):
    """
    Calculates average of 'neighbor_target' surrounding videos.
    Elastic Logic: Tries to get 5 before and 5 after.
    If not enough on one side, takes more from the other.
    """
    total_videos = len(all_videos)
    if total_videos <= 1:
        return None

    # 1. Define the ideal 'span' (Current video + neighbors)
    # If we want 10 neighbors, we need a span of 11 videos total.
    span_size = neighbor_target + 1

    # 2. Calculate ideal start point (centered)
    # For 10 neighbors, we want to start 5 slots back.
    half_window = neighbor_target // 2
    start_idx = current_index - half_window

    # 3. Elastic Adjustments
    # A. If start is negative (too new), shift window right to start at 0
    if start_idx < 0:
        start_idx = 0

    # B. Calculate end index based on start
    end_idx = start_idx + span_size

    # C. If end exceeds list (too old), shift window left to fit
    if end_idx > total_videos:
        end_idx = total_videos
        # Try to push start back to maintain span size
        start_idx = max(0, end_idx - span_size)

    # 4. Collect views from the window, SKIPPING the current video
    neighbor_views = []

    # Loop from start to end (exclusive)
    for i in range(start_idx, end_idx):
        if i == current_index:
            continue

        v = all_videos[i].get('view_count')
        if v is not None:
            neighbor_views.append(v)

    if not neighbor_views:
        return None

    return statistics.mean(neighbor_views)

def download_thumbnail(video_id, url):
    try:
        if not url:
            url = f"https://img.youtube.com/vi/{video_id}/maxresdefault.jpg"
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            with open(os.path.join(OUTPUT_FOLDER, f"{video_id}.jpg"), "wb") as f:
                f.write(response.content)
    except:
        pass

def main():
    videos = get_long_form_data(CHANNEL_ID)
    print(f"Found {len(videos)} videos. Processing...")

    with open(f'data/raw/{CHANNEL_ID}.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Video ID', 'Title', 'Current Views', 'Elastic Avg', 'V-Score'])

        for i, video in tqdm(enumerate(videos), total=len(videos)):
            vid_id = video.get('id')
            title = video.get('title')
            current_views = video.get('view_count')
            thumb_url = video.get('thumbnails', [{}])[-1].get('url')

            if not vid_id or current_views is None:
                continue

            # 1. Calculate Elastic Average
            elastic_avg = calculate_elastic_average(videos, i, NEIGHBOR_COUNT)

            # 2. Calculate Score
            if elastic_avg and elastic_avg > 0:
                v_score = current_views / elastic_avg # V-Score == Virality Score
                avg_str = f"{elastic_avg:.0f}"
                score_str = f"{v_score:.2f}"
            else:
                avg_str = "N/A"
                score_str = "N/A"

            # 3. Download & Save
            download_thumbnail(vid_id, thumb_url)

            writer.writerow([
                vid_id,
                title,
                current_views,
                avg_str,
                score_str
            ])

    print(f"\nDone! Data saved in '{CHANNEL_ID}.csv'")

if __name__ == "__main__":
    main()

In [10]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# --- CONFIGURATION ---
NICHES = ["technology", "programming", "software engineering"]
MAX_CHANNELS_PER_NICHE = 10  # How many results to fetch per keyword

with open("api.txt", "r") as f:
    API_KEY = f.read().strip()

def search_channels_by_niche(api_key, niches, max_results=10):
    youtube = build("youtube", "v3", developerKey=api_key)

    # We use a dictionary to automatically handle duplicates
    # Key = Channel ID, Value = Channel Title
    unique_channels = {}

    print(f"Starting search for {len(niches)} niches...\n")

    for niche in niches:
        print(f"🔎 Searching for: '{niche}'...")

        try:
            # The search().list() endpoint is the standard way to find channels
            request = youtube.search().list(
                part="snippet",
                type="channel",       # strictly search for channels
                q=niche,              # the search term
                maxResults=max_results,
                order="relevance"     # usually finds the most "popular" for that term
            )
            response = request.execute()

            for item in response.get('items', []):
                channel_id = item['snippet']['channelId']
                channel_title = item['snippet']['title']

                # LOGIC: Check for duplicates
                if channel_id not in unique_channels:
                    unique_channels[channel_id] = channel_title
                    print(f"   ✅ Added: {channel_title}")
                else:
                    print(f"   ⚠️ Duplicate ignored: {channel_title}")

        except HttpError as e:
            print(f"   ❌ API Error: {e}")

    return unique_channels

   ✅ Added: Modern Software Engineering
   ✅ Added: Software Engineering
   ✅ Added: Software Engineering
   ✅ Added: Software Engineering
   ✅ Added: Software Engineering Institute | Carnegie Mellon University
   ✅ Added: Software Engineering Daily
   ✅ Added: Software Engineering
   ✅ Added: Software Engineering - ALX
   ✅ Added: Software Developer Diaries
   ✅ Added: HNU Software Engineering

Final Collection: 45 Unique Channels
UCy0tKL1T7wFoYcxCe0xjN6Q : Technology Connections
UCUuvZ0skL2WYZ3rhPMZbfdA : Brick Technology
UCAL3JXZSzSm8AlZyD3nQdBA : Primitive Technology
UCrM7B7SL_g1edFOnmj-SDKg : Bloomberg Technology
UCYVU6rModlGxvJbszCclGGw : Rob Braxman Tech
UCf9phz2kU2DaJBwDATTD05Q : House Technology
UCKWaEZ-_VweaEx1j62do_vQ : IBM Technology
UCYUPLUCkMiUgiyVuluCc7tQ : Technology for Teachers and Students
UCiz26UeGvcTy4_M3Zhgk7FQ : Defog Tech
UCz6PEeVLG1TL6jMRTvSLm4g : Matt Talks Tech
UCKnaflu7tLEm2S5yHBYfmtg : Complete Technology
UC3ByF8DcZ3yxUs7VP1NOuyA : REWA Technology
UCF8H7dYH

In [11]:
channels = search_channels_by_niche(API_KEY, NICHES, MAX_CHANNELS_PER_NICHE)

dict_keys(['UCy0tKL1T7wFoYcxCe0xjN6Q', 'UCAL3JXZSzSm8AlZyD3nQdBA', 'UC4Xt1e33nqi6udIjVEJa8Nw', 'UCKWaEZ-_VweaEx1j62do_vQ', 'UCUuvZ0skL2WYZ3rhPMZbfdA', 'UCrM7B7SL_g1edFOnmj-SDKg', 'UCf9phz2kU2DaJBwDATTD05Q', 'UC9MQp8a5uhaIosZPHaoqEXQ', 'UCYUPLUCkMiUgiyVuluCc7tQ', 'UCYVU6rModlGxvJbszCclGGw', 'UCz6PEeVLG1TL6jMRTvSLm4g', 'UCKnaflu7tLEm2S5yHBYfmtg', 'UC4Tklxku1yPcRIH0VVCKoeA', 'UCiz26UeGvcTy4_M3Zhgk7FQ', 'UCF8H7dYHK6AvJF0EVonO3cw', 'UC3ByF8DcZ3yxUs7VP1NOuyA', 'UCFnKqecC7F2HTnWZlEzExbw', 'UC0QRG-gMqHhF5-BPdmJfs1w', 'UCRzScB0a-dc6OCYYZlP9qpA', 'UC7FWqacDz5A5bhbjxXbRN2Q', 'UCTR22zaU51pb2P72JS8oJtg', 'UC1tVU8H153ZFO9eRsxdJlhA', 'UC08uERSqTFS3xSwXKMLrUVg', 'UC905GJVaT_10oSdZq8sDoRg', 'UCZff6I2okeH71I5jsE0LirQ', 'UCuEOSK8blSM6j5jxVp3ttnQ', 'UCMN0a7GHQnC6H74SmCGSmdw', 'UCWv7vMbMWH4-V0ZXdmDpPBA', 'UCBsAE4_Cvr7FGnF6KVw9JfA', 'UCfJyQ3P2k_SuqfxVdqIEQNw', 'UCpKb02FsH4WH4X_2xhIoJ1A', 'UC42pOSNg804f1wCcj7qL0mA', 'UCV7cZwHMX_0vk8DSYrS7GCg', 'UCvGwM5woTl13I-qThI4YMCg', 'UCLZIElNyubHOvbfudT7KS1A', 'UCCfqyGl

In [31]:
channel_ids = list(channels.keys())
channel_ids += ['UCgKFOz_KrMbmypWrawtzDQg']
print(channel_ids.index('UCgKFOz_KrMbmypWrawtzDQg'))
print(len(channel_ids))
print(channel_ids)

45
46
['UCy0tKL1T7wFoYcxCe0xjN6Q', 'UCAL3JXZSzSm8AlZyD3nQdBA', 'UC4Xt1e33nqi6udIjVEJa8Nw', 'UCKWaEZ-_VweaEx1j62do_vQ', 'UCUuvZ0skL2WYZ3rhPMZbfdA', 'UCrM7B7SL_g1edFOnmj-SDKg', 'UCf9phz2kU2DaJBwDATTD05Q', 'UC9MQp8a5uhaIosZPHaoqEXQ', 'UCYUPLUCkMiUgiyVuluCc7tQ', 'UCYVU6rModlGxvJbszCclGGw', 'UCz6PEeVLG1TL6jMRTvSLm4g', 'UCKnaflu7tLEm2S5yHBYfmtg', 'UC4Tklxku1yPcRIH0VVCKoeA', 'UCiz26UeGvcTy4_M3Zhgk7FQ', 'UCF8H7dYHK6AvJF0EVonO3cw', 'UC3ByF8DcZ3yxUs7VP1NOuyA', 'UCFnKqecC7F2HTnWZlEzExbw', 'UC0QRG-gMqHhF5-BPdmJfs1w', 'UCRzScB0a-dc6OCYYZlP9qpA', 'UC7FWqacDz5A5bhbjxXbRN2Q', 'UCTR22zaU51pb2P72JS8oJtg', 'UC1tVU8H153ZFO9eRsxdJlhA', 'UC08uERSqTFS3xSwXKMLrUVg', 'UC905GJVaT_10oSdZq8sDoRg', 'UCZff6I2okeH71I5jsE0LirQ', 'UCuEOSK8blSM6j5jxVp3ttnQ', 'UCMN0a7GHQnC6H74SmCGSmdw', 'UCWv7vMbMWH4-V0ZXdmDpPBA', 'UCBsAE4_Cvr7FGnF6KVw9JfA', 'UCfJyQ3P2k_SuqfxVdqIEQNw', 'UCpKb02FsH4WH4X_2xhIoJ1A', 'UC42pOSNg804f1wCcj7qL0mA', 'UCV7cZwHMX_0vk8DSYrS7GCg', 'UCvGwM5woTl13I-qThI4YMCg', 'UCLZIElNyubHOvbfudT7KS1A', 'UCCfqyGl3nq_